### Generating A Drum Rhythm with NN

#### ToDo's

**17.05.2024** (Friday)
- Connnect to VS code! ✅
- Choose framework - Jax / PyTorch / Keras (not recommended for its simplicity) ✅
- Pick dataset ✅

    > Groove MIDI Dataset [Link](https://magenta.tensorflow.org/datasets/groove#download) + [Paper](https://arxiv.org/pdf/1905.06118)

- Pick model - RNN (+ MLP?) ✅
- Study the dataset and think of pre-processing it
- Study framework


**21.05.2024** (Tuesday)
- Define problem
- Define goal
- Start with setting up training model template


**23.05.2024** (Thursday)
- We code a model that acts as a generator: after running the code it should produce a simple beat pattern
- Keep things as simple as possible: use only one sounds for now
- Preprocessing and chosing the right format/encdoing is going to be the hardest part 
    - Time series: discretize, align all samples so they start on the same bar
    - Filter out different sounds (keep only one drum sounds)
    - 4/4 bar but leave space (32 time slices) for more complex pattern
    - Keep same duration for Feed-Forward Network or not for RNN
 - To keep it simple, maybe generate own data with the desired format
 - Regular meetings!!!
 - Keras is also fine
 - Understand the dataset really well before setting up the network


 **27.05.2024** (Monday)
 - Dateset?
 - Framework division
 - Data format
 - Feed-Forward or RNN
 - Regular Meetings


#### Pre-processing 

**Resources:**

> [Short IBM Article: Data preprocessing in detail](https://developer.ibm.com/articles/data-preprocessing-in-detail/)  
> 
> [Medium Detailed Article with Example: Data pre-processing: A step-by-step guide](https://towardsdatascience.com/data-pre-processing-a-step-by-step-guide-541b083912b5)
>
> [Easy Kaggle Article: Machine Learning Episode #1: Data Pre-processing steps using Python](https://www.kaggle.com/discussions/getting-started/151612)
>
> 





#### Frameworks

**JAX**

> [Extensive Article with Example for RNN, MLP, and CNN: Getting started with JAX (MLPs, CNNs & RNNs)](https://roberttlange.com/posts/2020/03/blog-post-10/)
